In [323]:
import sys
sys.path.append('/Users/Jessica/Desktop/Coding/School/2802ICT/Assignment2')

from imports import *
from collections import Counter

In [324]:
class Node:     #this class creats an object for each node in the tree

    def __init__(self, feature=None, thresh=None, left=None, right=None, *, value=None):        
        self.feature = feature
        self.value = value
        self.right_child = right
        self.left_child = left
        self.thresh = thresh
        self.labels = []

    def isNode(self):
        if self.value is not None:
            return True
        else:
            return False

In [325]:

class DecisionTree: 

    def __init__(self, data, max_depth, min_split = 2, num_feats = None):
        self. df = data

        self.feature_train = []
        self.feature_test = []
        self.target_train = []
        self.target_test = []

        self.max_depth = max_depth
        self.min_split = min_split
        #given dataset has 6 features but can use datasets
        self.num_feats = num_feats 
        self.root = None

    def driver(self, split):

        self.split_data(split)
        self.fit(self.feature_train, self.target_train)

        prediction_train = self.predict(self.feature_train)
        
        percent_train = self.accuracy(self.target_train, prediction_train)

    def predict(self, features):
        return np.array([self.traverse(x, self.root) for x in features])       # traverses the tree to find the best splits

    def traverse(self, x, node):
        #if it is a node return its value
        if node.isNode():
            #return the value           
            return node.value
         # x is the current value
         # feature train, it will recursively search for the split and 
         # threshold over the children to find the most optimal splits   
        if x[node.feature] <= node.thresh:                  
            return self.traverse(x, node.left_child)
        return self.traverse(x, node.right_child)                      
    
    def accuracy(self, actual, prediction):
        res = self.confusion_matrix(actual, prediction)
        self.calc(res)
        #accuracy = (self.counter(actual, prediction))/len(actual)
        #return accuracy

    def confusion_matrix(self, actual, prediction):
        classes = np.unique(actual)
        K = len(classes)
        result = np.zeros((K, K))

        for i in range(len(actual)):
            
            if actual[i] == "unacc":
                index = 0
                if prediction[i] == "unacc":
                    result[index][0] += 1
                elif prediction[i] == "acc":
                    result[index][1] += 1
                elif prediction[i] == "good":
                    result[index][2] += 1
                elif prediction[i] == "vgood":
                    result[index][3] += 1
            
            elif actual[i] == 'acc':
                index = 1
                if prediction[i] == 'unacc':
                    result[index][0] += 1
                elif prediction[i] == 'acc':
                    result[index][1] += 1
                elif prediction[i] == 'good':
                    result[index][2] += 1
                elif prediction[i] == 'vgood':
                    result[index][3] += 1

            elif actual[i] == 'good':
                index = 2
                if prediction[i] == 'unacc':
                    result[index][0] += 1
                elif prediction[i] == 'acc':
                    result[index][1] += 1
                elif prediction[i] == 'good':
                    result[index][2] += 1
                elif prediction[i] == 'vgood':
                    result[index][3] += 1

            elif actual[i] == 'vgood':
                index = 3
                if prediction[i] == 'unacc':
                    result[index][0] += 1
                elif prediction[i] == 'acc':
                    result[index][1] += 1
                elif prediction[i] == 'good':
                    result[index][2] += 1
                elif prediction[i] == 'vgood':
                    result[index][3] += 1
     
        return result

    def calc(self, result):
        
        size = len(result)
        locations = np.zeros((size, size))
        tp = tn = fp = fn = 0

        for i in range(size):
            if locations[i][i] != 1:
                tp += result[i][i]
                locations[i][i] = 1
            for j in range(size): 
                if locations[i][j] != 1:
                    fp += (result[i][0] + result[i][1] + result[i][2] + result[i][3] - result[i][i])
                    locations[i][0] = locations[i][1] = locations[i][2] = locations[i][3] = 1
            for j in range(size):    
                if locations[i][j] != 1:
                    fn += (result[0][i] + result[1][i] + result[2][i] + result[3][i] - result[i][i])
                    locations[0][i] = locations[1][i] = locations[2][i] = locations[3][i] = 1
            for j in range(size):    
                if locations[i][j] != 1:
                    tn += result[i][j]
                    locations[i][j] = 1
            
            print(locations)
            locations = np.zeros((size, size))

        print( tp, tn, fp, fn)


        p = (tp)/(tp+fp)
        print("Precision: ",p)
        r = (tp)/(tp+fn)
        print("Recall: ",r)
        f1 = 2/((1/p)+(1/r))
        print("F1 Score: ",f1)

             
    def fit(self, feature_train, target_train):

        self.num_feats = feature_train.shape[1]
        self.root = self.build_tree(feature_train, target_train)

    #builds the tree recursively
    def build_tree(self, f_train, t_train, depth = 0):
        
        #amount of samples in dataset
        num_samples = f_train.shape[0]
        #amount of features
        num_feats = self.num_feats
        #num of labels -> (acc/unacc/good/vgood)
        n_labels = len(np.unique(t_train))
        #criteria that end the searching
        if (depth >= self.max_depth or n_labels == 1 or num_samples < self.min_split):
            leaf = self.high_freq(t_train)             
            return Node(value = leaf)
        
        feature_ids = np.random.choice(num_feats, self.num_feats, replace=False)
        
        if f_train.size != 0 and t_train.size != 0:
            # finds the best feature to split on and the threshold
            best_feature, threshold = self.best_split(f_train, t_train, feature_ids)
            left_id, right_id = self.next_split(f_train[:, best_feature], threshold)

            left = self.build_tree(f_train[left_id,:], t_train[left_id], depth + 1)
            right = self.build_tree(f_train[right_id,:], t_train[right_id], depth + 1)

        return Node(best_feature, threshold, left, right)
     
    def best_split(self, f_train, t_train, f_ids):
        
        best_gain = -1
        split_id, split_threshold = None, None
        
        for f_id in f_ids:
            #f_id_col holds the column for 1 feature at a time
            f_id_col = f_train[:, f_id]

            #thresholds has the unique elements in each feature column
            thresholds = np.unique(f_id_col)

            for threshold in thresholds:
                gain = self.info_gain(t_train, f_id_col, threshold)

                if gain > best_gain:

                    best_gain = gain
                    split_id = f_id
                    split_threshold = threshold

        return split_id, split_threshold

    def info_gain(self, t_train, feature_col, threshold):
        #finds the entropy of the current set
        parent_entropy = self.entropy(t_train)
        #finds the left and right id values based on split
        left_id, right_id = self.next_split(feature_col, threshold)

        if len(left_id) == 0 or len(right_id) == 0:
            return 0

        n = len(t_train)
        l_len, r_len = len(left_id), len(right_id)
        l_entropy, r_entropy = self.entropy(t_train[left_id]), self.entropy(t_train[right_id])
        child_entropy = (l_len/n) * l_entropy + (r_len/n) * r_entropy
        info_gain_val = parent_entropy - child_entropy

        return info_gain_val

    def next_split(self, feature_col, threshold):
        left_id = np.argwhere(feature_col <= threshold).flatten()
        right_id = np.argwhere(feature_col > threshold).flatten()
        return left_id, right_id
    
    #finds the entropy -> fequency/length 
    def entropy(self, t_train):
        histogram = []
        counted_vals = Counter(t_train)
        for word in set(self.target_train):
            new_val = counted_vals[word]
            histogram.append(new_val)

        histogram = np.array(histogram)
        length = len(t_train)              
        calc = histogram/length
        
        return -(np.sum([c * np.log2(c) for c in calc if c > 0]))

    def split_data(self, split):
        dataset = np.array(self.df)
        #randomly shuffle dataset
        np.random.shuffle(dataset)  

        #multiply length by wanted percent and // 1 to ensure its a wh0le number
        test_size = int((len(dataset)*split)//1)                      
        train_size = int(len(dataset) - test_size)

        #use the length to make respective lists
        train = dataset[:train_size]                                     
        test = dataset[test_size:]

        #only want the last column. THEN map -> join as cars are strings
        target_train = np.delete(train, np.s_[0:6], axis = 1)                  
        self.target_train = np.array(list(map(''.join, target_train)))

        self.feature_train = np.delete(train, 6, axis = 1)                  
        #self.feature_train = np.array(list(map(''.join, feature_train)))

        self.target_test = np.delete(test, np.s_[0:6], axis = 1)
        #self.target_test = np.array(list(map(''.join, target_test)))

        self.feature_test = np.delete(test, 6, axis = 1)
        #self.feature_test = np.array(list(map(''.join, feature_test)))

    def high_freq(self, target_train):   
        counter = Counter(target_train)  
        #most common value present which is useful when determing where to split  
        most_common = counter.most_common(1)[0][0] 
        return most_common

In [326]:

if __name__ == "__main__":

    #file is read from a class called SetUp
    data = SetUp("car.csv", {'buying':'string','maint':'string','doors':'string','persons':'string','lug_boot':'string','safety':'string','class':'string'}) 

    ####################### USER SETUP AREA #######################
    
    split = 0.25
    max_depth = 5
    print_each_k_percentage = False             #if you want to print each k's percentage    
    
    ####################### END OF USER SETUP #######################

    DT = DecisionTree(data.df, max_depth)
    DT.driver(split)





[[1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 1. 1. 1.]]
1127.0 0 169.0 0
Precision:  0.8695987654320988
Recall:  1.0
F1 Score:  0.9302517540239373
